# AutoGen FAQs for Customer Support with Snowflake Cortex

## Overview

This notebook demonstrates how to automatically generate FAQ entries from customer support ticket data using Snowflake's Cortex AI capabilities. The notebook leverages Snowflake's Cortex features and SQL to process customer conversations and create structured FAQ content.

## Prerequisites

- Access to a Snowflake environment with Cortex enabled

## Data Exploration

The notebook includes a preliminary step to examine the structure of customer support tickets. This is done through a simple SQL query that shows sample data from the CUSTOMER_SUPPORT_TICKETS table

In [ ]:
Select ISSUE_AREA, ISSUE_CATEGORY, CONVERSATION FROM CUSTOMER_SUPPORT_TICKETS LIMIT 3;

## FAQ Generation Process

The core of the notebook is a SQL query that:

1. **Groups Conversations**: Combines related conversations by issue area and category
2. **Generates Prompts**: Creates AI prompts using the grouped conversations
3. **Processes Output**: Handles the AI-generated responses and formats them into structured FAQ entries
4. **Stores Results**: Inserts the processed FAQs into the CUSTOMER_SUPPORT_FAQ table

Key features of the FAQ generation:
- Uses Anthropic Claude 3.5 Sonnet model for generating responses
- Implements JSON formatting for structured output
- Includes error handling for JSON parsing
- Maintains metadata about issue areas and categories

In [ ]:
INSERT INTO CUSTOMER_SUPPORT_FAQ (ISSUE_AREA, ISSUE_CATEGORY, QUESTION, ANSWER)
WITH GroupedData AS (
    SELECT 
        ISSUE_AREA, 
        ISSUE_CATEGORY, 
        LISTAGG(CONVERSATION, ' ||| ') AS CONCATENATED_CONVERSATIONS
    FROM CUSTOMER_SUPPORT_TICKETS
    GROUP BY ISSUE_AREA, ISSUE_CATEGORY
),
Prompts AS (
    SELECT 
        ISSUE_AREA, 
        ISSUE_CATEGORY,
        LEFT(
            CONCAT(
                'You are a customer service assistant. Based on the following conversations, generate an FAQ entry in valid JSON format with two keys: "question" and "answer". Conversations: ',
                CONCATENATED_CONVERSATIONS,
                ' Output: {"question": "<your question here>", "answer": "<your answer here>"}'
            ),
            2000 -- Limit the prompt length
        ) AS PROMPT
    FROM GroupedData
),
RawOutputs AS (
    SELECT 
        ISSUE_AREA, 
        ISSUE_CATEGORY,
        SNOWFLAKE.CORTEX.COMPLETE('claude-3-5-sonnet', PROMPT) AS RAW_OUTPUT
    FROM Prompts
),
CleanedOutputs AS (
    SELECT 
        ISSUE_AREA, 
        ISSUE_CATEGORY,
        REPLACE(
            REPLACE(
                REPLACE(
                    RAW_OUTPUT,
                    '```', '' -- Remove backticks
                ),
                'json', '' -- Remove "json" prefix
            ),
            '\n', '' -- Remove newlines
        ) AS CLEANED_OUTPUT
    FROM RawOutputs
),
ParsedOutputs AS (
    SELECT 
        ISSUE_AREA,
        ISSUE_CATEGORY,
        TRY_PARSE_JSON(CLEANED_OUTPUT) AS PARSED_JSON -- Parse cleaned JSON
    FROM CleanedOutputs
    WHERE TRY_PARSE_JSON(CLEANED_OUTPUT) IS NOT NULL -- Ensure valid JSON parsing
)
SELECT 
    ISSUE_AREA, 
    ISSUE_CATEGORY,
    PARSED_JSON:"question"::STRING AS QUESTION, -- Extract the question
    PARSED_JSON:"answer"::STRING AS ANSWER -- Extract the answer
FROM ParsedOutputs;


## Results Review

The final step allows viewing the generated FAQs through a simple query

In [ ]:
SELECT * FROM CUSTOMER_SUPPORT_FAQ ORDER BY CREATED_AT DESC;